LIBRARY

In [ ]:
import requests
import pandas as pd
import requests
import time
import urllib
from bs4 import BeautifulSoup
import json
import csv
from functools import *

***INTERNAL FUNCTION***

In [ ]:
 ### Find value of key in nested json 
def find_value_of_key(value, key):
    val = None
    if isinstance(value,dict):
        if key in value:
            return value[key]
        for i in value.items():
            val = find_value_of_key(i[1], key)
            if val is not None:
                return val
    if isinstance(value,list):
        for i in value:
            val = find_value_of_key(i, key)
    return val

**CRAWLING CATEGORIES FROM ONE PAGE**

In [ ]:
def get_product_catagories(url):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    response = requests.get(url=url, headers=http_headers)
    parser = BeautifulSoup(response.text, 'html.parser', )
    product_box = parser.findAll('script')
    product_catagories = []
    for product in product_box:
        try:
            item = json.loads(str(product.contents[0]))
        except:
            item = {}
        product_val = find_value_of_key(item,"breadcrumbs")
        if product_val is not None:
            [product_catagories.append(i) for i in product_val if i["category_id"] != 0] 
    return product_catagories


### Testing the Functional
get_product_catagories('https://tiki.vn/dien-thoai-samsung-galaxy-z-flip-4-8gb-128gb-hang-chinh-hang-p194034217.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.163088_Y.1744616_Z.3253348_CN.Product-Ads-B4Q4&itm_medium=CPC&itm_source=tiki-ads&spid=194034225')


#GET ID OF PRODUCT

In [ ]:
def get_max_page_number(name:str, categories_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url = f'https://tiki.vn/{name}/c{categories_id}'
    response = None
    while(response is None):
        response = requests.get(url=url, headers=http_headers)
    parser = BeautifulSoup(response.text, 'html.parser', )

    # find all 
    div_tag = parser.find("div", {"data-view-id":"product_list_pagination_container"})   
    ul_tag = div_tag.find('ul')
    li_tag = ul_tag.findAll('li')
    def find_max_tag(a,b):
        if isinstance(a,int):
            a_num = a
        else:
            a_tag = a.find('a')
            a_num = a_tag.get('data-view-label') if a_tag.get('data-view-label') is not None else 0
        b = b.find('a')
        b_num = b.get('data-view-label') if b.get('data-view-label') is not None else 0
        return max(int(a_num), int(b_num))
    
    return reduce(find_max_tag,li_tag)

### Testing the Functional
get_max_page_number("Điện thoại Smartphone", 1795)

** CRAWLING ALL ID FROM CATEGORIES **

In [29]:
def crawling_product_id(name:str, categories_id:int, page_number:int):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    product_list = set()
    for i in range(page_number):
        url =f'https://tiki.vn/{name}/c{categories_id}?page{i}'
        response = None
        while(response is None):
            response = requests.get(url=url, headers=http_headers)
        parser = BeautifulSoup(response.text, 'html.parser', )
    
        # find all 
        product_box = parser.findAll('a')
        if (len(product_box) == 0):
            pass
        for product in product_box:
            data = product.get("data-view-content")
            if data is not None:
                item = json.loads(data)
                id = item['click_data'].get('id')
                if id is not None:
                    product_list.add(id)
    return list(product_list)

### Testing the Functional
crawling_product_id("Điện thoại Smartphone", 1795,12)

[197553025,
 177746939,
 155536275,
 184051220,
 197216279,
 177343384,
 170581659,
 53090589,
 197670818,
 170580388,
 194034217,
 127372598,
 114092342,
 203764665,
 88100671,
 197214015,
 175967811,
 57809866,
 184059211,
 164923596,
 197214029,
 168282061,
 128865871,
 164937168,
 201548628,
 160801751,
 203864283,
 101615454,
 158714974,
 166034912,
 184036446,
 135332196,
 189658085,
 187254759,
 193310183,
 170497385,
 142545270,
 175845367,
 132068858,
 114766715]

**CALL API TO GET PRODUCT INFORMATION, CATEGORIES**

In [31]:
def get_data(product_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url = f'https://tiki.vn/api/v2/products/{product_id}'
    response = requests.get(url, headers = http_headers)
    result = response.json()
    categories = result.get('breadcrumbs')
    return response.json(), categories
    
data, categories = get_data(177746939)
print(categories)


[{'url': '/dien-thoai-may-tinh-bang/c1789', 'name': 'Điện Thoại - Máy Tính Bảng', 'category_id': 1789}, {'url': '/dien-thoai-smartphone/c1795', 'name': 'Điện thoại Smartphone', 'category_id': 1795}, {'url': 'https://tiki.vn/dien-thoai-samsung-galaxy-a73-5g-8gb-128gb-hang-chinh-hang-da-kich-hoat-bao-hanh-dien-tu-p177746939.html', 'name': 'Điện Thoại Samsung Galaxy A73 5G (8GB/128GB) - Hàng chính hãng - ĐÃ KÍCH HOẠT BẢO HÀNH ĐIỆN TỬ', 'category_id': 0}]


**DUMP DATA TO FILE AND EXTRACT CATEGORIES**

In [37]:
### DATA FEILD 
f_brand = ['id','name','slug'] ##### brand
f_brand_crawl = [{'brand':f_brand}]
f_seller = ['id','name','store_id','is_best_store'] ##### current_seller, other_seller
f_seller_crawl = [{'current_seller': f_seller},{'other_seller': f_seller}]
f_product = ['id','name', 'short_url', 'short_description', 'price', 'list_price', 'original_price', 'discount', 'discount_rate',
            'rating_average', 'inventory_status', 'inventory_type', 'is_fresh', 'seller','is_gift_card', 'is_gift_card','data_version',
            {'brand':['id']}, {'current_seller': ['id','price']},{'other_seller': ['id','price']},'category_id','fulfillment_type',
            'amplitude']
f_categories = ['url', 'name', 'category_id']

In [38]:
#### DUMP HEADER TO FILE
def convert_product_feild(feild):
    res = []
    for x in feild:
        if isinstance(x,dict):
            for k,v in x.items():
                mini_res = [k + i for i in v]
            res = res + mini_res
        else:
            res.append(x)
    return res

product_header = convert_product_feild(f_product)
with open('Data/product.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(product_header)

with open('Data/categories.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(f_categories)

with open('Data/sellers.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(f_seller)

In [ ]:
http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
}
response = requests.get(url="https://tiki.vn/dien-thoai-samsung-galaxy-z-flip-4-8gb-128gb-hang-chinh-hang-p194034217.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.163088_Y.1744616_Z.3253348_CN.Product-Ads-B4Q4&itm_medium=CPC&itm_source=tiki-ads&spid=194034225", headers=http_headers)
parser = BeautifulSoup(response.text, 'html.parser' )
with open('data.txt', 'w') as file:
    data = file.write(response.content.decode())
print(data)